# 数据集类

神经网络需要从海量数据中学习。

我们将把海量数据封装进**数据集**（Dataset）类中，包括训练数据和测试数据，并支持批处理。

In [61]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

为了让张量可以处理批量样本（二维数组），我们需要修改一下大小（size）属性，只返回最后一个维度的数值。就是单个样本包含的的特征值数量。

In [62]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    @property
    def size(self):
        return self.data.shape[-1]

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础数据集

基础数据集是一个抽象类。封装了加载和遍历数据需要的一些函数和接口：

* **加载**（load）：加载数据的虚拟接口；
* **训练**（train）：切换到模型训练模式；
* **测试**（eval）：切换到模型测试模式；
* **所有样本**（items）：当前模式下（训练、或者测试）所有样本；
* **样本**（getitem）：根据索引（index）读取（批）样本。

两个新的属性：

* **样本形状**（shape）：特征值数量，和标签值数量；
* **样本数量**（len）：当前模式下（训练、或者推理）样本数量。

创建一个数据集，我们需要知道：

* **批大小**（batch_size）：每次读取样本的数量。

In [63]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    @property
    def shape(self):
        return Tensor(self.features).size, Tensor(self.labels).size

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [64]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    @property
    def parameters(self):
        return []

    def __repr__(self):
        return ''

### 基础损失函数

In [65]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

我们对优化器进行一些升级，来适应模型训练多次迭代的要求：
* **重置**（reset）：每次迭代开始前，清零所有梯度，准备开始下一轮梯度计算。

In [66]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

## 数据

### 数据集

这是一个最简版本的数据集，加载了我们在第一部分使用的，小明冰激凌店的数据（天气预报，和冰激凌销量）。

In [67]:
class Dataset(Dataset):

    def load(self):
        self.train_features = [[22.5, 72.0],
                               [31.4, 45.0],
                               [19.8, 85.0],
                               [27.6, 63]]
        self.train_labels = [[95],
                             [210],
                             [70],
                             [155]]

        self.test_features = [[28.1, 58.0]]
        self.test_labels = [[165]]

## 模型

### 线性层

In [68]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data
            self.bias.grad += np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        return p

    @property
    def parameters(self):
        return [self.weight, self.bias]

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（均方误差）

在损失函数类，我们需要将损失项除以批大小，以避免造成梯度叠加。

In [69]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data) / len(y.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

In [70]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

## 设置

### 学习率

In [71]:
LEARNING_RATE = 0.00001

### 批大小

In [72]:
BATCH_SIZE = 2

## 训练

### 建模

In [73]:
# 数据集
dataset = Dataset(BATCH_SIZE)
# 层
layer = Linear(dataset.shape[0], dataset.shape[1])
print(layer)
# 损失函数
loss_fn = MSELoss()
# 优化器
optimizer = SGDOptimizer(layer.parameters, lr=LEARNING_RATE)

Linear[weight(1, 2); bias(1,)]


### 迭代

我们可以开始使用数据集来进行小批次迭代训练。

需要注意的一点是，每次迭代的梯度计算前，我们要先使用优化器清零梯度，避免把上一次迭代的梯度叠加到本次迭代。

In [74]:
for i in range(len(dataset)):
    # 依次读入训练数据：特征、标签
    features, labels = dataset[i]

    # 前向传播
    predictions = layer(features)
    # 计算损失
    loss = loss_fn(predictions, labels)
    # 清空梯度
    optimizer.reset()
    # 梯度计算
    loss.backward()
    # 反向传播
    optimizer.step()

## 验证

### 推理

进入数据集测试模式，就可以开始使用测试数据验证模型。

In [75]:
# 进入测试模式
dataset.eval()
# 读入测试数据
features, labels = dataset.items()

predictions = layer(features)
print(f'prediction: {predictions}')

prediction: Tensor([[56.19176426]])


### 评估

In [76]:
loss = loss_fn(predictions, labels)
print(f'loss: {loss}')

loss: Tensor(11839.232164432306)
